In [ ]:
!pip install earthengine-api
!pip install geemap
!pip install scikit-learn
!pip install folium
!pip install matplotlib
!pip install pandas
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import ee
import geemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import folium
from datetime import datetime, timedelta
import time

# Initialize Earth Engine
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize(project='ee-deepti')

# Define region of interest (Delhi NCR)
roi = ee.Geometry.Rectangle([76.8, 28.4, 77.8, 29.0])

# Set date range (last 90 days)
end_date = datetime.now() - timedelta(days=30)
start_date = end_date - timedelta(days=90)  # Extended from 60 to 90 days for more data

end_date_str = end_date.strftime('%Y-%m-%d')
start_date_str = start_date.strftime('%Y-%m-%d')

print(f"Analyzing data from {start_date_str} to {end_date_str}")

# Get Sentinel-5P NO2 data
s5p = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2') \
    .filterBounds(roi) \
    .filterDate(start_date_str, end_date_str) \
    .select('tropospheric_NO2_column_number_density')

# Check if we have data
s5p_count = s5p.size().getInfo()
print(f"Number of Sentinel-5P images in collection: {s5p_count}")
if s5p_count == 0:
    raise ValueError("No Sentinel-5P images found for the specified region and time period. Please adjust your ROI or date range.")

# Calculate mean NO2
no2_mean = s5p.mean()

# Get resolution info
s5p_resolution = no2_mean.projection().nominalScale().getInfo()
print(f"Original S5P NO2 data resolution: {s5p_resolution} meters")

# Get Sentinel-2 data
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate(start_date_str, end_date_str) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(lambda img: img.clip(roi))

# Define function to add indices
def add_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')
    ndwi = image.normalizedDifference(['B3', 'B11']).rename('NDWI')
    return image.addBands([ndvi, ndbi, ndwi])

# Apply indices and create composite
s2 = s2.map(add_indices)
s2_composite = s2.median()

# Select bands from S2
s2_selected = s2_composite.select(['NDVI', 'NDBI', 'NDWI', 'B2', 'B3', 'B4', 'B8', 'B11', 'B12'])

# Get DEM data
dem = ee.Image('USGS/SRTMGL1_003').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope')
aspect = ee.Terrain.aspect(dem).rename('aspect')
dem = dem.addBands([slope, aspect]).rename(['elevation', 'slope', 'aspect'])

# Get landcover data
landcover = ee.ImageCollection('ESA/WorldCover/v200').first().clip(roi).rename('landcover')

# Get population data
population = ee.ImageCollection('JRC/GHSL/P2023A/GHS_POP') \
    .filterDate('2020-01-01', '2021-01-01') \
    .first() \
    .clip(roi) \
    .rename('population')

# Get nightlights data
nightlights = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
    .filterDate(start_date_str, end_date_str) \
    .select('avg_rad') \
    .mean() \
    .clip(roi) \
    .rename('nightlights')

# Define target resolution
TARGET_RESOLUTION = 100  # meters

print(f"Original S5P NO2 resolution: ~7000 meters")
print(f"Target resolution: {TARGET_RESOLUTION} meters")
print(f"Resolution improvement factor: {int(s5p_resolution/TARGET_RESOLUTION)}x")

# Resample all layers to appropriate resolutions
no2_resampled = no2_mean.reproject(crs='EPSG:4326', scale=s5p_resolution)
s2_resampled = s2_selected.reproject(crs='EPSG:4326', scale=TARGET_RESOLUTION)
dem_resampled = dem.reproject(crs='EPSG:4326', scale=TARGET_RESOLUTION)
landcover_resampled = landcover.reproject(crs='EPSG:4326', scale=TARGET_RESOLUTION)
population_resampled = population.reproject(crs='EPSG:4326', scale=TARGET_RESOLUTION)
nightlights_resampled = nightlights.reproject(crs='EPSG:4326', scale=TARGET_RESOLUTION)

# Combine all data layers
training_image = ee.Image.cat([
    no2_resampled.rename('NO2'),
    s2_resampled,
    dem_resampled,
    landcover_resampled,
    population_resampled,
    nightlights_resampled
])

# Generate random training points
training_points = ee.FeatureCollection.randomPoints(roi, 5000, seed=42)  # Increased from 3000 to 5000 points

# Sample data at training points
training_data = training_image.sampleRegions(
    collection=training_points,
    scale=TARGET_RESOLUTION,
    geometries=True,
    tileScale=16
)

# Extract training data
print("Extracting training data samples...")
try:
    training_data_list = training_data.getInfo()['features']
    training_df = pd.DataFrame([feature['properties'] for feature in training_data_list])
except Exception as e:
    print(f"Error extracting training data: {e}")
    print("Trying with fewer samples and higher tileScale...")
    # Retry with fewer points and higher tileScale
    training_points = ee.FeatureCollection.randomPoints(roi, 2000, seed=42)
    training_data = training_image.sampleRegions(
        collection=training_points,
        scale=TARGET_RESOLUTION,
        geometries=True,
        tileScale=32
    )
    training_data_list = training_data.getInfo()['features']
    training_df = pd.DataFrame([feature['properties'] for feature in training_data_list])

print(f"Collected {len(training_df)} training samples")

# Clean data
original_count = len(training_df)
training_df = training_df.dropna()
print(f"Removed {original_count - len(training_df)} samples with missing values")

# Prepare features and target
feature_columns = [col for col in training_df.columns if col != 'NO2']
X = training_df[feature_columns]
y = training_df['NO2']

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
print("Training Random Forest model for spatial resolution enhancement...")
rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

rf_model.fit(X_train, y_train)

# Evaluate model
y_pred = rf_model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
rmse = np.sqrt(mse)

print(f"\nModel Performance:")
print(f"Mean Squared Error: {mse:.8f}")
print(f"Root Mean Squared Error: {rmse:.8f}")
print(f"R² Score: {r2:.4f}")

# Plot validation results
plt.figure(figsize=(10, 6))
plt.scatter(y_val, y_pred, alpha=0.5)
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], 'r--')
plt.xlabel('Actual NO2 Values')
plt.ylabel('Predicted NO2 Values')
plt.title('Random Forest Model: Actual vs. Predicted NO2 Values')
plt.grid(True)
plt.tight_layout()
plt.savefig('model_validation.png')
plt.close()  # Close instead of plt.show() to avoid blocking execution

# Calculate feature importance
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 8))
plt.barh(feature_importance['Feature'][:15], feature_importance['Importance'][:15])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Top 15 Features for NO2 Spatial Downscaling')
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()  # Close instead of plt.show()

print("Top 10 Most Important Features:")
print(feature_importance.head(10))

# Create Earth Engine model
print("\nCreating Earth Engine-compatible model for spatial downscaling...")

def create_ee_rf_model():
    top_features = feature_importance['Feature'][:15].tolist()

    classifier = ee.Classifier.smileRandomForest(**{
        'numberOfTrees': 200,
        'minLeafPopulation': 2,
        'seed': 42
    }).setOutputMode('REGRESSION')

    filtered_training_data = training_data.select(['NO2'] + top_features)

    trained_classifier = classifier.train(
        features=filtered_training_data,
        classProperty='NO2',
        inputProperties=top_features
    )
    return trained_classifier, top_features

ee_model, top_features = create_ee_rf_model()

# Create high-resolution prediction
print(f"Creating high-resolution prediction image at {TARGET_RESOLUTION}m resolution...")

predictors_high_res = ee.Image.cat([
    s2_resampled,
    dem_resampled,
    landcover_resampled,
    population_resampled,
    nightlights_resampled
]).select(top_features)

predicted_no2_highres = predictors_high_res.classify(ee_model).rename('predicted_NO2')

# Create Sentinel-2 RGB for visualization
s2_rgb = s2_composite.select(['B4', 'B3', 'B2']).divide(10000)

# Visualization parameters
no2_vis_params = {
    'min': 0.00005,
    'max': 0.0002,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'orange', 'red']
}

# === VISUALIZATION SECTION ===
print("\nPreparing multiple visualization options...")

# ------ METHOD 1: SINGLE MAP VIEW ------
try:
    print("\nMETHOD 1: Creating single map with all layers...")
    m = geemap.Map()
    m.centerObject(roi, 10)
    m.add_basemap('HYBRID')
    m.addLayer(no2_mean, no2_vis_params, 'Coarse NO2 (~7km resolution)')
    m.addLayer(predicted_no2_highres, no2_vis_params, f'Fine Resolution NO2 ({TARGET_RESOLUTION}m)')
    m.addLayer(s2_rgb, {'min': 0, 'max': 0.3}, 'Sentinel-2 RGB', False)
    m.addLayerControl()
    print("Displaying single map view...")
    display(m)
except Exception as e:
    print(f"Error in single map view: {e}")

# ------ METHOD 2: SPLIT MAP APPROACH 1 ------
try:
    print("\nMETHOD 2: Creating split map (approach 1)...")
    split_map1 = geemap.Map()
    split_map1.centerObject(roi, 10)
    split_map1.add_basemap('HYBRID')

    # Add layers
    split_map1.addLayer(no2_mean, no2_vis_params, 'Original NO2 (~7km)')
    split_map1.addLayer(predicted_no2_highres, no2_vis_params, f'Enhanced NO2 ({TARGET_RESOLUTION}m)')

    # Try to get layer IDs from the map object
    try:
        left_layer_id = list(split_map1.layers)[-2].id
        right_layer_id = list(split_map1.layers)[-1].id
        split_map1.split_map(left_layer=left_layer_id, right_layer=right_layer_id)
    except (AttributeError, IndexError) as e:
        print(f"Could not get layer IDs from object: {e}")
        # Fallback to indices
        split_map1.split_map(left_index=len(split_map1.layers)-2, right_index=len(split_map1.layers)-1)

    print("Displaying split map (approach 1)...")
    display(split_map1)
except Exception as e:
    print(f"Error in split map approach 1: {e}")

# ------ METHOD 3: SPLIT MAP APPROACH 2 ------
try:
    print("\nMETHOD 3: Creating split map (approach 2)...")
    split_map2 = geemap.Map()
    split_map2.centerObject(roi, 10)
    split_map2.add_basemap('HYBRID')

    # Add the layers
    split_map2.addLayer(no2_mean, no2_vis_params, 'Original NO2 (~7km)')
    split_map2.addLayer(predicted_no2_highres, no2_vis_params, f'Enhanced NO2 ({TARGET_RESOLUTION}m)')

    # Try to find the layer names
    try:
        layer_names = [layer.name for layer in split_map2.layers if hasattr(layer, 'name')]
        if len(layer_names) >= 2:
            split_map2.split_map(left_name=layer_names[-2], right_name=layer_names[-1])
    except Exception as e:
        print(f"Could not get layer names: {e}")
        # Fallback to direct approach
        split_map2.split_map(left_name='Original NO2 (~7km)', right_name=f'Enhanced NO2 ({TARGET_RESOLUTION}m)')

    print("Displaying split map (approach 2)...")
    display(split_map2)
except Exception as e:
    print(f"Error in split map approach 2: {e}")

# ------ METHOD 4: TWO SEPARATE MAPS ------
try:
    print("\nMETHOD 4: Creating two separate maps...")

    # Map 1: Original NO2
    map1 = geemap.Map()
    map1.centerObject(roi, 10)
    map1.add_basemap('HYBRID')
    map1.addLayer(no2_mean, no2_vis_params, 'Coarse NO2 (~7km resolution)')
    map1.addLayer(s2_rgb, {'min': 0, 'max': 0.3}, 'Sentinel-2 RGB', False)
    map1.addLayerControl()

    # Map 2: Enhanced NO2
    map2 = geemap.Map()
    map2.centerObject(roi, 10)
    map2.add_basemap('HYBRID')
    map2.addLayer(predicted_no2_highres, no2_vis_params, f'Fine Resolution NO2 ({TARGET_RESOLUTION}m)')
    map2.addLayer(s2_rgb, {'min': 0, 'max': 0.3}, 'Sentinel-2 RGB', False)
    map2.addLayerControl()

    # Display both maps
    print("Displaying original NO2 map...")
    display(map1)
    print("Displaying enhanced NO2 map...")
    display(map2)
except Exception as e:
    print(f"Error in separate maps: {e}")

# ------ METHOD 5: FOLIUM DIRECT APPROACH ------
try:
    print("\nMETHOD 5: Using folium directly for visualization...")

    # Get the center of the ROI
    roi_center = [28.7, 77.3]  # Approximate center of Delhi NCR

    # Create a folium map
    folium_map = folium.Map(location=roi_center, zoom_start=10)

    # Add Google Satellite basemap
    folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Satellite',
        overlay=False,
        control=True
    ).add_to(folium_map)

    # Add the NO2 layers using geemap's ee_tile_layer utility
    try:
        # Try to get the geemap tile layer utility
        from geemap.core import ee_tile_layer

        # Add coarse NO2 layer
        coarse_tile = ee_tile_layer(no2_mean, no2_vis_params, 'Coarse NO2 (~7km resolution)')
        coarse_tile.add_to(folium_map)

        # Add enhanced NO2 layer
        enhanced_tile = ee_tile_layer(predicted_no2_highres, no2_vis_params, f'Enhanced NO2 ({TARGET_RESOLUTION}m)')
        enhanced_tile.add_to(folium_map)
    except Exception as e:
        print(f"Could not add Earth Engine layers to folium map: {e}")

    # Add layer control
    folium.LayerControl().add_to(folium_map)

    # Display the map
    print("Displaying folium map...")
    display(folium_map)
except Exception as e:
    print(f"Error in folium direct approach: {e}")

# ------ METHOD 6: CREATE A TILELAYER MAP MANUALLY ------
try:
    print("\nMETHOD 6: Creating tile layer manually...")

    # Function to get map ID and token
    def get_tile_url(ee_image_object, vis_params):
        map_id = ee_image_object.getMapId(vis_params)
        tile_url = map_id['tile_fetcher'].url_format
        return tile_url

    # Create a basic map
    manual_map = folium.Map(location=[28.7, 77.3], zoom_start=10)

    # Add Google Satellite basemap
    folium.TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Satellite',
        overlay=False,
        control=True
    ).add_to(manual_map)

    # Try to add the NO2 layers manually
    try:
        # Get tile URLs
        coarse_url = get_tile_url(no2_mean, no2_vis_params)
        enhanced_url = get_tile_url(predicted_no2_highres, no2_vis_params)

        # Add tile layers
        folium.TileLayer(
            tiles=coarse_url,
            attr='Earth Engine',
            name='Coarse NO2 (~7km resolution)',
            overlay=True,
            control=True
        ).add_to(manual_map)

        folium.TileLayer(
            tiles=enhanced_url,
            attr='Earth Engine',
            name=f'Enhanced NO2 ({TARGET_RESOLUTION}m)',
            overlay=True,
            control=True
        ).add_to(manual_map)
    except Exception as e:
        print(f"Could not create tile layers manually: {e}")

    # Add layer control
    folium.LayerControl().add_to(manual_map)

    # Display the map
    print("Displaying manual tile layer map...")
    display(manual_map)
except Exception as e:
    print(f"Error in manual tile layer approach: {e}")

# Export to Drive
def export_to_drive(image, description, scale, region=None):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        scale=scale,
        region=roi.getInfo()['coordinates'],
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
    task.start()
    print(f"Started export task for {description} at {scale}m resolution")

print("\nExporting data to Google Drive...")
export_to_drive(no2_mean, 'coarse_resolution_NO2', s5p_resolution)
export_to_drive(predicted_no2_highres, 'fine_resolution_NO2', TARGET_RESOLUTION)

# Calculate additional metrics
explained_variance = 1 - (np.var(y_val - y_pred) / np.var(y_val))
mean_abs_error = np.mean(np.abs(y_val - y_pred))

print("\nAdditional Validation Metrics:")
print(f"Explained Variance: {explained_variance:.4f}")
print(f"Mean Absolute Error: {mean_abs_error:.8f}")

# Plot error distribution
errors = y_val - y_pred
plt.figure(figsize=(10, 6))
plt.hist(errors, bins=50, alpha=0.7)
plt.axvline(x=0, color='r', linestyle='--')
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.title('Distribution of Spatial Downscaling Errors')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('error_distribution.png')
plt.close()  # Close instead of plt.show()

print("\nSpatial Resolution Enhancement Complete!")
print(f"Enhanced NO2 map resolution from {s5p_resolution}m to {TARGET_RESOLUTION}m")
print(f"Resolution improvement factor: {int(s5p_resolution/TARGET_RESOLUTION)}x")

Analyzing data from 2025-07-11 to 2025-10-09
Number of Sentinel-5P images in collection: 1271
Original S5P NO2 data resolution: 111319.49079327357 meters
Original S5P NO2 resolution: ~7000 meters
Target resolution: 100 meters
Resolution improvement factor: 1113x
Extracting training data samples...
Collected 4987 training samples
Removed 0 samples with missing values
Training Random Forest model for spatial resolution enhancement...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    6.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.1s finished



Model Performance:
Mean Squared Error: 0.00000000
Root Mean Squared Error: 0.00000294
R² Score: 0.4439
Top 10 Most Important Features:
      Feature  Importance
0         B11    0.211708
2          B2    0.121587
10  elevation    0.110855
1         B12    0.095638
6        NDBI    0.075981
13      slope    0.055784
5          B8    0.051796
8        NDWI    0.049724
9      aspect    0.043636
3          B3    0.042240

Creating Earth Engine-compatible model for spatial downscaling...
Creating high-resolution prediction image at 100m resolution...

Preparing multiple visualization options...

METHOD 1: Creating single map with all layers...
Displaying single map view...


Map(center=[28.700632280422344, 77.29999999999988], controls=(WidgetControl(options=['position', 'transparent_…


METHOD 2: Creating split map (approach 1)...
Could not get layer IDs from object: 'EELeafletTileLayer' object has no attribute 'id'
Displaying split map (approach 1)...


...

Displaying enhanced NO2 map...


Map(center=[28.700632280422344, 77.29999999999988], controls=(WidgetControl(options=['position', 'transparent_…


METHOD 5: Using folium directly for visualization...
Could not add Earth Engine layers to folium map: cannot import name 'ee_tile_layer' from 'geemap.core' (/usr/local/lib/python3.12/dist-packages/geemap/core.py)
Displaying folium map...



METHOD 6: Creating tile layer manually...
Displaying manual tile layer map...



Exporting data to Google Drive...
Started export task for coarse_resolution_NO2 at 111319.49079327357m resolution
Started export task for fine_resolution_NO2 at 100m resolution

Additional Validation Metrics:
Explained Variance: 0.4439
Mean Absolute Error: 0.00000186

Spatial Resolution Enhancement Complete!
Enhanced NO2 map resolution from 111319.49079327357m to 100m
Resolution improvement factor: 1113x
